In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy 
from neurosim.models.ssr import StateSpaceRealization as SSR
import sys

In [3]:
sys.path.append('../..')

In [4]:
from subspaces import SubspaceIdentification, factorize
from em import StableStateSpaceML

In [5]:
from dca_research.lqg import LQGComponentsAnalysis as LQGCA
from dca_research.kca import KalmanComponentsAnalysis as KCA

In [ ]:
# Demonstrate what closing the loop with this analysis looks like
# (0) Can one obtain the minimum phase model from a small state dimension?
# (1) Sweep through the state dimensions and fit SSID/StableML (and when feasible, VAR models). Use the fitted models to produce correlation sequences relevant
# for the control problem. Assess consistency with solution derived from the ground truth correltaions
# (2) Compare the LQGCA solutions compared with the ordinary optimal feedback control problem
# (3) Does underestimating the state dimension cause problems? 

In [6]:
state_dim = 40
obs_dim = 40
A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))
while max(np.abs(np.linalg.eigvals(A))) > 0.99:
    A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))

#C = scipy.stats.ortho_group.rvs(state_dim)[:, 0:obs_dim].T
ssr = SSR(A=A)

TypeError: __init__() missing 2 required positional arguments: 'B' and 'C'

In [6]:
y = ssr.trajectory(10000)

In [10]:
ssid = SubspaceIdentification()
A, C, Cbar, L0, Q, R, S = ssid.identify(y, order=20)

In [21]:
B, D = factorize(A, C, Cbar, L0)

In [22]:
K = B @ np.linalg.inv(D)

In [26]:
# This was done in innovations form, so there likely aren't any huge prolems, but what if we start from the traidtional state space form? Does the model necessarily
# give rise to system zeros? Can one obtain the innovations form from the low rank model?

In [27]:
# Specific observables:
# (1) Transmission zeros
# (4) Actuator mismatch

In [7]:
# (1) Finding transmission zeros
def transmission_zeros(A, B, C, D):
    # Form matrices for generalized eigenvalue problem
    try:
        BB = np.block([[np.eye(A.shape[0]), np.zeros(B.shape)], [np.zeros(C.shape), np.zeros(D.shape)]])
    except:
        pdb.set_trace()
    AA = np.block([[A, B], [C , D]])
    eig, _ = scipy.linalg.eig(AA,b=BB)
    return eig

In [8]:
def autocorrelation1(A, C, Q, R, S, T):

    autocorr = np.zeros((T, C.shape[0], C.shape[0]))

    P = scipy.linalg.solve_discrete_lyapunov(A, Q)

    autocorr[0, ...] = C @ P @ C.T + R

    # Construct Cbar from A, P, C, and S
    Cbar = C @ P @ A.T + S.T

    for i in range(1, T):
        autocorr[i, ...] = C @ np.linalg.matrix_power(A, i - 1) @ Cbar.T
    return autocorr 


In [9]:
def autocorrelation2(A, C, Cbar, Q, R, T):

    autocorr = np.zeros((T, C.shape[0], C.shape[0]))

    P = scipy.linalg.solve_discrete_lyapunov(A, Q)

    autocorr[0, ...] = C @ P @ C.T + R

    for i in range(1, T):
        autocorr[i, ...] = C @ np.linalg.matrix_power(A, i - 1) @ Cbar.T
    return autocorr 

In [10]:
import pdb

In [27]:
model_reps = 5
trajectory_reps = 1
state_dim = 40
obs_dim = 40

results_list = []

for i in range(model_reps):

    A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))
    while max(np.abs(np.linalg.eigvals(A))) > 0.99:
        A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))

    #C = scipy.stats.ortho_group.rvs(state_dim)[:, 0:obs_dim].T
    ssr = SSR(A=A, B = np.eye(A.shape[0]), C = np.eye(A.shape[0]))

    # Fit LQGCA and KCA (time reversed) to the true autocorrelations
    lqgca = LQGCA(T=3, d=2)
    lqgca.cross_covs = ssr.autocorrelation(5)
    # Fit 10 tries
    true_lqgca_coefs = []
    for _ in range(10):
        v, _ = lqgca._fit_projection()
        true_lqgca_coefs.append(v)
    # Reverse time KCA loadings
    kcamodel = KCA(T=3, d=2, causal_weights=(0, 1))
    kcamodel.cross_covs = ssr.autocorrelation(5)
    true_kcacoefs = []
    for _ in range(10):
        v, _ = kcamodel._fit_projection()
        true_kcacoefs.append(v)

    for j in range(trajectory_reps):
        y = ssr.trajectory(int(1e4))
        # First, fit SSID and state space models where we tune the model order from too low, right, to too high
        for model_order in [20, 25, 30, 35, 40]:
            print('Fitting SSID')
            ssid = SubspaceIdentification()
            ssid.identify(y, order=model_order)
            A, C, Cbar, L0, Q, R, S = ssid.identify(y, order=model_order)        
            Sigma0 = scipy.linalg.solve_discrete_lyapunov(A, Q)

            # Calculate system zeros - experiment with the factorize function and also sqrt factorizing R and Q
            B1, D1 = factorize(A, C, Cbar, L0)
            eB, uB = scipy.linalg.eig(Q)
            eD, uD = scipy.linalg.eig(R)

            # Rectify spectrum
            eB[np.abs(eB) < 1e-8] = 0
            eD[np.abs(eD) < 1e-8] = 0

            B2 = uB @ np.diag(np.sqrt(eB))
            D2 = uD @ np.diag(np.sqrt(eD))

            z1 = transmission_zeros(A, B1, C, D1)
            # z2 = transmission_zeros(A, B2, C, D2)

            ssid_cross_covs = autocorrelation2(A, C, Cbar, Q, R, 5)

            # LQGCA loadings
            lqgca = LQGCA(T=3, d=2)
            lqgca.cross_covs = ssid_cross_covs
            # Fit 10 tries
            lqgca_coefs = []
            for _ in range(10):
                v, _ = lqgca._fit_projection()
                lqgca_coefs.append(v)
            # Reverse time KCA loadings
            kcamodel = KCA(T=3, d=2, causal_weights=(0, 1))
            kcamodel.cross_covs = ssid_cross_covs
            kcacoefs = []
            for _ in range(10):
                v, _ = kcamodel._fit_projection()
                kcacoefs.append(v)


            result = {}
            result['model_rep'] = 1
            result['trajectory_rep'] = 1
            result['model_order'] = model_order
            result['true_lqgca'] = true_lqgca_coefs
            result['true_kca'] = true_kcacoefs
            result['z1'] = z1
            result['lqgca_ssid'] = lqgca_coefs
            result['kca_ssid'] = kcacoefs


            print('Fitting SSML')
            ssm = StableStateSpaceML(max_iter=5, init_strategy='manual')
            try:
                ssm.fit(y, state_dim=model_order, Ainit=A, Cinit=C, Qinit=Q, Rinit=R,
                        x0 = np.zeros(A.shape[0]), Sigma0 = Sigma0)
            except:
                result['lgqca_ssl'] = np.nan
                result['kca_ssl'] = np.nan
                results_list.append(result)
                continue

            ccm_ml = autocorrelation1(ssm.A, ssm.C, np.eye(ssm.A.shape[0]), ssm.R, np.zeros((ssm.A.shape[0], ssm.C.shape[0])), 5)

            # LQGCA loadings
            lqgca = LQGCA(T=3, d=2)
            lqgca.cross_covs = ccm_ml
            # Fit 10 tries
            lqgca_coefs = []
            for _ in range(10):
                v, _ = lqgca._fit_projection()
                lqgca_coefs.append(v)
            # Reverse time KCA loadings
            kcamodel = KCA(T=3, d=2, causal_weights=(0, 1))
            kcamodel.cross_covs = ccm_ml
            kcacoefs = []
            for _ in range(10):
                v, _ = kcamodel._fit_projection()
                kcacoefs.append(v)

            result['lqgca_ssl'] = lqgca_coefs
            result['kca_ssl'] = kcacoefs

            results_list.append(result)


Fitting SSID
Fitting SSML
E step: 9.753644
M step: 0.598059
Fitting SSID
Fitting SSML
E step: 11.911794
M step: 0.462713


In [16]:
# Tasks for tomorrow
# Produce plots for this guy, its VAR equivalent, and similarly the VAR equivalent for the other guy
# However, start with running ARMA state space on a VAR fit
# To do control theoretic analysis on non-square models, may need to "ignore" the D matrix
# Produce conceptual figure about the role of predictor spaces
# Time permitting, outline polar decomposition stuff (unlikely)